In [21]:
import torch
from torch import nn

In [22]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0952],
        [-0.0764]], grad_fn=<AddmmBackward0>)

In [23]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2697,  0.1452, -0.2148, -0.1194, -0.2325, -0.1529, -0.1769,  0.1363]])), ('bias', tensor([-0.2238]))])


In [24]:
X = torch.rand(size=(1, 4))
net(X)

tensor([[-0.0870]], grad_fn=<AddmmBackward0>)

In [25]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2697,  0.1452, -0.2148, -0.1194, -0.2325, -0.1529, -0.1769,  0.1363]])), ('bias', tensor([-0.2238]))])


In [26]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.4460, -0.4863,  0.4415,  0.2093],
        [-0.0518, -0.3731, -0.4680, -0.2652],
        [-0.2709, -0.4537, -0.0052,  0.2898],
        [-0.4598,  0.2015,  0.1378, -0.2840],
        [ 0.3950, -0.3028, -0.0086,  0.2376],
        [-0.0775, -0.0011, -0.1760, -0.1202],
        [-0.4958, -0.2441,  0.3843, -0.1608],
        [-0.1779, -0.1609, -0.1812, -0.2588]])), ('bias', tensor([ 0.4397,  0.3501,  0.3740, -0.2452, -0.2400,  0.4595,  0.3003, -0.2234]))])


In [27]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2238], requires_grad=True)
tensor([-0.2238])


In [28]:
net[2].weight.grad == None

True

In [29]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [30]:
net.state_dict()['2.bias'].data

tensor([-0.2238])

In [31]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3698]], grad_fn=<AddmmBackward0>)

In [32]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [33]:
rgnet[0][1][0].bias.data

tensor([-0.1874, -0.0493, -0.4585, -0.3703,  0.4965, -0.1007, -0.1732, -0.3224])

In [34]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0128, -0.0082, -0.0061, -0.0030]), tensor(0.))

In [35]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [37]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)    

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.3628,  0.6354,  0.2952, -0.1561],
        [ 0.5744,  0.2733,  0.0373, -0.5005],
        [-0.0551,  0.0642, -0.4437,  0.1578],
        [-0.5503, -0.2286, -0.1852, -0.1310],
        [ 0.1493,  0.6223,  0.1534,  0.7003],
        [ 0.3818, -0.0007, -0.2338, -0.6223],
        [-0.6783, -0.0860, -0.6793,  0.3422],
        [ 0.0790,  0.3758,  0.3580,  0.5962]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [38]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.5261,  9.4363,  0.0000, -0.0000],
        [-0.0000, -0.0000,  0.0000, -9.3401]], grad_fn=<SliceBackward0>)

In [41]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0, 1] = 38
net[0].weight.data

tensor([[42.0000, 38.0000,  3.0000,  3.0000],
        [ 3.0000,  3.0000,  3.0000, -6.3401],
        [ 3.0000, -4.0604, 12.5083, -6.8293],
        [-3.6690,  8.4233,  3.0000,  3.0000],
        [-6.5231,  3.0000,  3.0000,  3.0000],
        [ 3.0000,  9.5859, 11.0948,  3.0000],
        [ 3.0000, 12.5939,  3.0000,  3.0000],
        [ 3.0000,  3.0000, -5.1097, -5.0248]])

In [46]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data == net[4].weight.data)

tensor([True, True, True, True, True, True, True, True])
tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])
